# Download Datset and Understand the Format

In [1]:
import pandas as pd
# Collect the data from the zipped files
df_training = pd.read_csv('kddcup.data.gz', header=None)
df_testing = pd.read_csv('corrected.gz', header=None)
df_training_10 = pd.read_csv('kddcup.data_10_percent.gz', header=None)

In [2]:
# Split the data from labels
trlabels = df_training.iloc[:, 41].values
tslabels = df_testing.iloc[:, 41].values
trlabels_10 = df_training_10.iloc[:, 41].values
training = df_training.drop(df_training.columns[41], axis=1)
testing = df_testing.drop(df_testing.columns[41], axis=1)
training_10 = df_training_10.drop(df_training_10.columns[41], axis=1)
# The data after dropping the headers should be of shape (4898431, 41) and (311029, 41)

assert (training.shape == (4898431, 41))
assert (testing.shape == (311029, 41))
assert (training_10.shape == (494021, 41))
print(trlabels)
print(tslabels)
print(trlabels_10)

['normal.' 'normal.' 'normal.' ... 'normal.' 'normal.' 'normal.']
['normal.' 'normal.' 'normal.' ... 'normal.' 'normal.' 'normal.']
['normal.' 'normal.' 'normal.' ... 'normal.' 'normal.' 'normal.']


# Convert the categorical values into numeric values.

In [3]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

def cat_to_num(trcolumn, tscolumn):
    """
    Converts 2 categorical columns of the same types into numerical columns

    Args:
        trcolumn (ndarray): ndarray of values of the first column.
        tscolumn (ndarray): ndarray of values of the second column.

    Returns:
        tuple: a tuple of 2 ndarrays
    """
    encoder = LabelEncoder()
    categories = set(np.unique(trcolumn)).union(set(np.unique(tscolumn)))
    encoder.fit(list(categories))
    return encoder.transform(trcolumn), encoder.transform(tscolumn)

In [4]:
# Copy the data into another dataframe to convert its categorical values into numerical.
num_training = training.copy()
num_testing = testing.copy()
num_training_10 = training_10.copy()
num_testing_10 = testing.copy()
# Convert the categorical features.
for i in range(1, 4):
    values = cat_to_num(num_training.iloc[:, i].values, num_testing.iloc[:, i].values)
    num_training.isetitem(i, values[0])
    num_testing.isetitem(i, values[1])


for i in range(1, 4):
    values = cat_to_num(num_training_10.iloc[:, i].values, num_testing_10.iloc[:, i].values)
    num_training_10.isetitem(i, values[0])
    num_testing_10.isetitem(i, values[1])
# Convert the labels.
num_trlabels, num_tslabels = cat_to_num(trlabels, tslabels)
num_trlabels_10, num_tslabels_10 = cat_to_num(trlabels_10, tslabels)

In [5]:
num_training.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,0,1,24,9,215,45076,0,0,0,0,...,0,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
1,0,1,24,9,162,4528,0,0,0,0,...,1,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0
2,0,1,24,9,236,1228,0,0,0,0,...,2,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0
3,0,1,24,9,233,2032,0,0,0,0,...,3,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0
4,0,1,24,9,239,486,0,0,0,0,...,4,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0


The data is now available in two forms:

Form One (Categorical):

training

*   trlabels
*   tslabels
*   training
*   testing

Form Two (Numerical):

*   trlabels
*   tslabels
*   training
*   testing



# Clustering Using K-Means 

In [6]:
def k_means(X, k, epsilon):
    
    n_samples, n_features = X.shape
    
    # Randomly choose k data points as the initial centroids
    centroids = X[np.random.choice(n_samples, k, replace=False)]
    distances = np.zeros((n_samples, k))
    labels = np.zeros(n_samples)
    old_centroids = np.zeros((k, n_features))
    
    # Continue until the centroids don't change by more than epsilon
    while np.linalg.norm(centroids - old_centroids) > epsilon:
        old_centroids = centroids.copy()
        
        # Calculate the Euclidean distances from each sample to each centroid
        for i in range(k):
            distances[:, i] = np.linalg.norm(X - centroids[i], axis=1)
        
        # Assign each sample to the nearest centroid
        labels = np.argmin(distances, axis=1)
        
        # Update the centroids to be the mean of the samples assigned to them
        for i in range(k):
            X_i = X[labels == i]
            if len(X_i) == 0:
                centroids[i] = old_centroids[i]
            else:
                centroids[i] = np.mean(X_i, axis=0)
        
    return centroids

In [ ]:
from sklearn.metrics.cluster import contingency_matrix
from scipy.optimize import linear_sum_assignment

labels = []
train = np.array(num_training_10)
test = np.array(num_testing_10)
results = []
for K in [7,15,23,31,45]:
    centroids = k_means(train, K, 0.001)
    distances = np.linalg.norm(test[:, np.newaxis, :] - centroids, axis=2)
    labels = np.argmin(distances, axis=1)
    contingency = contingency_matrix(num_tslabels_10, labels)
    row_ind, col_ind = linear_sum_assignment(-contingency)
    y_pred = np.zeros_like(labels)
    for i, j in zip(row_ind, col_ind):
        y_pred[labels == j] = i
    print("K:",K)       
    print("precision:",precision(y_pred.tolist(), num_tslabels.tolist()))
    print("recall",recall(y_pred.tolist(), num_tslabels.tolist()))
    print("f1_score",f1(y_pred.tolist(), num_tslabels.tolist()))
    print("conditional_entropy",conditional_entropy(y_pred.tolist(), num_tslabels.tolist()))
    print()

K: 7
precision: 0.5277546466728182
recall 0.9998039726904134
cluster: 0 pre: 0.5276780890638265   rec: 1.0   f1: 0.6908236661130512
cluster: 1 pre: 1.0   rec: 0.00019804267819715149   f1: 0.00039600693012127716
cluster: 35 pre: 1.0   rec: 1.650355651642929e-05   f1: 3.300656830709311e-05
cluster: 16 pre: 0.9285714285714286   rec: 0.0006436387041407423   f1: 0.001286385750803991
cluster: 27 pre: 0.6666666666666666   rec: 6.601422606571717e-05   f1: 0.00013201537979174574
f1_score 0.1385342161484151
conditional_entropy 2.0068936057817357

K: 15
precision: 0.7577653530699711
recall 0.7399829403933673
cluster: 0 pre: 0.9902730437305098   rec: 0.6657220688520394   f1: 0.7961939191626914
cluster: 1 pre: 0.5054631828978622   rec: 0.9690346083788707   f1: 0.6643771464252263
cluster: 2 pre: 1.0   rec: 9.902133909857574e-05   f1: 0.0001980230696876186
cluster: 35 pre: 0.6666666666666666   rec: 6.601422606571717e-05   f1: 0.00013201537979174574
cluster: 3 pre: 1.0   rec: 1.650355651642929e-05   f

# Normalized Cut

In [7]:
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split as tts
import numpy as np

np.random.seed(42)


def vecsort(vectors, values):
    """
    Sorts vectors based on values.

    Args:
        vectors (nparray): nparray of vectors to be sorted.
        values (nparray): nparray of values to be used to sort.

    Returns:
        nparray: nparray of sorted vectors with respect to values.
    """
    return vectors[:, np.argsort(values)[::]]


def norm(data):
    """
    Normalizes a data matrix.

    Args:
        data (nparray): array of numbers.

    Returns:
        normalized: nparray of the normalized array result after normalizing the data matrix.
    """
    normalized = []
    for row in data:
        normalized.append(row / np.linalg.norm(row))
    return np.array(normalized)


def ncut(training_data, k):
    """
    Splits the data into a training set and testing set with ratio 0.15% for training dataset,
    then applies the normalized cut algorithm on the reduced training dataset.

    Args:
        data (pd.DataFrame): pd.DataFrame containing the original dataset.
        k (int): number of clusters.

    Returns:
        predicted, true: nparrays of predicted labels after applying the normalized cut algorithm and the true labels.
    """
    training = tts(training_data, random_state=42, train_size=0.0015)[0]
    
    # Get the true labels
    true = training.iloc[:, 41].values

    # Convert the data into numpy arrays
    training = np.array(training)

    # Construct the similarity graph
    S = cosine_similarity(training)

    # Construct the degree matrix
    degrees = np.sum(S, axis=1)
    D = np.diag(degrees)

    # Compute Laplacian Matrix
    L = D - S

    # Compute sorted eigenvectors of the Laplacian Matrix then normalize them
    values, vectors = np.linalg.eigh(L)
    eigvectors = vecsort(vectors, values)

    normalized = norm(eigvectors[:, :k])

    # Perform K-means clustering on eigenvectors
    centroids = k_means(normalized, k, 0.01)
    distances = np.linalg.norm(normalized[:, np.newaxis, :] - centroids, axis=2)
    predicted = np.argmin(distances, axis=1)

    return predicted, true


In [8]:
num_training['labels'] = num_trlabels
ncut_clustering = ncut(num_training, 11)
ncut_predicted = np.array(ncut_clustering[0])
ncut_true = np.array(ncut_clustering[1])

In [15]:
print("Precision:", precision(ncut_predicted.tolist(), ncut_true.tolist()))
print("Recall", recall(ncut_predicted.tolist(), ncut_true.tolist()))
print("F1 Measure",f1(ncut_predicted.tolist(), ncut_true.tolist()))
print("Conditional Entropy",conditional_entropy(ncut_predicted.tolist(), ncut_true.tolist()))

Precision: 0.9052674561045325
Recall 0.6246432642308797
cluster: 0 pre: 0.8695652173913043   rec: 0.013917884481558803   f1: 0.0273972602739726
cluster: 1 pre: 0.9955947136563876   rec: 0.15727209464161448   f1: 0.2716346153846154
cluster: 2 pre: 0.9895287958115183   rec: 0.1315240083507307   f1: 0.23218673218673222
cluster: 3 pre: 0.963963963963964   rec: 0.1489213639526792   f1: 0.2579867389993972
cluster: 4 pre: 0.94   rec: 0.09812108559498957   f1: 0.17769376181474478
cluster: 5 pre: 0.9756980351602895   rec: 0.8848769050410317   f1: 0.9280708225746956
cluster: 6 pre: 0.8610421836228288   rec: 0.44430217669654287   f1: 0.5861486486486486
cluster: 7 pre: 0.7046109510086456   rec: 0.11465416178194607   f1: 0.19721718088324258
cluster: 8 pre: 1.0   rec: 0.0048712595685455815   f1: 0.009695290858725763
cluster: 9 pre: 0.7666963490650045   rec: 0.5512163892445583   f1: 0.641340782122905
cluster: 10 pre: 1.0   rec: 0.025052192066805846   f1: 0.048879837067209775
F1 Measure 0.307113788255

# Evaluation

In [10]:
def clusterize(pred_labels, true_labels):
    if len(pred_labels) != len(true_labels):
        raise ValueError("The two list should be equal")
    clusters_set = set(pred_labels)
    # num_clusters = len(set(pred_labels))
    clusters = {}
    for cluster in clusters_set:
        clusters[cluster] = []
    for i in range(len(pred_labels)):
        clusters[pred_labels[i]].append(true_labels[i])
    return clusters, clusters_set

In [11]:
def precision(pred_labels, true_labels):
    clusters, clusters_set = clusterize(pred_labels, true_labels)
    res = 0
    for cluster in clusters_set:
        most_common = max(set(clusters[cluster]), key = clusters[cluster].count)
        count = clusters[cluster].count(most_common)
        res += (len(clusters[cluster]) / len(true_labels)) * (count / len(clusters[cluster]))
    return res

In [12]:
def recall(pred_labels, true_labels):
    clusters, clusters_set = clusterize(pred_labels, true_labels)
    res = 0
    r = len(clusters_set)
    for cluster in clusters_set:
        most_common = max(set(clusters[cluster]), key = clusters[cluster].count)
        count = clusters[cluster].count(most_common)
        count_total = true_labels.count(most_common)
        res += (len(clusters[cluster]) / len(true_labels)) * (count / count_total)
    return res

In [13]:
def f1(pred_labels, true_labels):
    clusters, clusters_set = clusterize(pred_labels, true_labels)
    res = 0
    r = len(clusters_set)
    for cluster in clusters_set:
        most_common = max(set(clusters[cluster]), key = clusters[cluster].count)
        count = clusters[cluster].count(most_common)
        count_total = true_labels.count(most_common)
        precision =  count / len(clusters[cluster])
        recall = count / count_total
        f1 = (2 * precision * recall) / (precision + recall)
        print(f"cluster: {cluster} pre: {precision}   rec: {recall}   f1: {f1}")
        res += (float(f1) / float(r))
    return res

In [14]:
from math import log2
def conditional_entropy(pred_labels, true_labels):
    clusters, clusters_set = clusterize(pred_labels, true_labels)
    res = 0
    true_labels_set = set(true_labels)
    for cluster in clusters_set:
        temp = 0
        for t in true_labels_set:
            t_count = clusters[cluster].count(t)
            if t_count != 0:
                temp += -(t_count / len(clusters[cluster])) * log2(t_count / len(clusters[cluster]))
        res += (len(clusters[cluster]) / len(true_labels)) * temp
    return res

# DBSCAN

In [6]:
vis = []
my_dict = {}
def dbscan(data, eps, min_samples):
    X = data.values
    global vis
    global my_dict
    vis = [0] * len(X)
    my_dict = {i: [] for i in range(len(X))}
    labels = [0] * len(X)
    cluster_id = 0
    for i in range(len(X)):
        if labels[i] != 0:
            continue            
        # Find all neighbors of the current point within eps distance
        neighbors = get_neighbors(X, i, eps)       
        # If the point is not a core point, mark it as an outlier
        if len(neighbors) < min_samples:
            labels[i] = -1
            continue       
        # Expand the cluster starting from the current core point
        cluster_id += 1
        labels[i] = cluster_id
        
        expand_cluster(X, labels, i, neighbors, eps, min_samples, cluster_id)
    
    return labels

def expand_cluster(X, labels, i, neighbors, eps, min_samples, cluster_id):
    """
    This function expands cluster for the ith 
    """
    # Loop over each neighbor of the core point
    for j in neighbors:
        if labels[j] == -1:
            labels[j] = cluster_id
        elif labels[j] == 0:
            labels[j] = cluster_id        
            # Find all neighbors of the current point within eps distance
            new_neighbors = get_neighbors(X, j, eps)        
            # If the point is a core point, add its neighbors to the list of neighbors
            if len(new_neighbors) >= min_samples:
                neighbors += new_neighbors

def get_neighbors(X, i, eps):
    """
    Thsis functions gets the neighbour for ith instance within given epsilon
    """
    global vis
    global my_dict
    if vis[i] == 1:
        return my_dict[i]
    neighbors = []
    for j in range(len(X)):
        if i == j:
            continue
            
        dist = np.linalg.norm(X[i] - X[j])
        if dist <= eps:
            neighbors.append(j)
    vis[i] = 1
    my_dict[i] = neighbors
    return neighbors

In [7]:
from sklearn.model_selection import train_test_split
num_training['labels'] = num_trlabels
train_data_copy, test_data = train_test_split(num_training, train_size=0.0015, random_state=42)
train_labels = np.array(train_data_copy.iloc[:, -1])
print(f"Training set size: {len(train_data_copy)}")

Training set size: 7347


In [8]:
pred_labels = dbscan(train_data_copy, 15, 3)
print(pred_labels)

[1, 2, 1, 2, 1, 3, 1, -1, 1, 1, 3, 3, 1, 1, -1, 3, 1, 1, 2, 1, 1, 1, 2, 1, 3, 4, 3, 3, 2, -1, 3, 1, 1, 3, 3, -1, 1, 3, -1, -1, 1, 1, 1, 1, 1, 5, 1, 1, -1, 3, 2, -1, -1, 1, 1, -1, 1, 3, -1, 3, 3, 1, 1, 3, 1, -1, -1, 1, 2, 2, 1, -1, 3, 1, -1, 3, 3, 1, 1, -1, 1, 1, 2, -1, -1, 3, 2, 1, 1, 3, 1, 1, 1, 3, 1, 3, 3, -1, 1, 3, 1, 3, 1, 3, 1, 1, 2, 1, -1, -1, 3, 3, -1, 1, 1, 1, -1, 1, 1, 1, 3, 1, 3, 2, 3, -1, 1, 2, -1, 3, 1, 3, 1, 3, 3, -1, 1, 1, 1, 3, -1, -1, 6, 1, 1, 3, -1, 3, 2, -1, 3, 1, -1, 3, 1, 1, 1, -1, 2, 1, 1, 1, 3, 1, 3, 1, -1, 1, -1, 3, -1, 4, 3, 1, 3, 1, 3, 2, 3, 2, 1, 4, 1, -1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 2, 2, 1, 3, 2, -1, 1, -1, 1, 1, 3, 1, -1, 3, 2, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 3, 1, 1, 3, -1, 1, 2, 2, 2, -1, 1, 3, 2, 1, 2, 2, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, -1, 3, 2, 1, 3, 3, -1, 1, -1, 1, 3, 3, 1, -1, 17, 1, 4, -1, 1, 3, 7, 2, 3, -1, 1, 2, -1, 1, 3, 1, -1, 1, -1, 4, 1, 1, 1, -1, 3, 1, 3, 6, -1, 3, -1, -1, 3, 1, 1, 1, -1, 3, 8, 1, 1, 1, 1, 2, 1, 1, -1, 2, -1, -1, -1, 

In [11]:
train_labels

array([27, 27, 27, ..., 27, 27, 27])

In [18]:
print("Precision:", precision(pred_labels, train_labels.tolist()))
print("Recall", recall(pred_labels, train_labels.tolist()))
print("F1 Measure",f1(pred_labels, train_labels.tolist()))
print("Conditional Entropy",conditional_entropy(pred_labels, train_labels.tolist()))

Precision: 0.9897917517354023
Recall 0.7563745809565455
cluster: 1 pre: 1.0   rec: 0.8112543962485346   f1: 0.8957928802588997
cluster: 2 pre: 1.0   rec: 0.18640093786635403   f1: 0.31422924901185767
cluster: 3 pre: 0.980806142034549   rec: 0.9814340588988476   f1: 0.98112
cluster: 4 pre: 1.0   rec: 0.05636743215031315   f1: 0.10671936758893281
cluster: 5 pre: 1.0   rec: 0.008350730688935281   f1: 0.016563146997929604
cluster: 6 pre: 1.0   rec: 0.014613778705636743   f1: 0.028806584362139915
cluster: 7 pre: 1.0   rec: 0.037578288100208766   f1: 0.07243460764587525
cluster: 8 pre: 1.0   rec: 0.8823529411764706   f1: 0.9375
cluster: 9 pre: 1.0   rec: 0.009742519137091163   f1: 0.019297036526533425
cluster: 10 pre: 1.0   rec: 0.023660403618649965   f1: 0.04622705642420122
cluster: 11 pre: 1.0   rec: 0.003479471120389701   f1: 0.00693481276005548
cluster: 12 pre: 1.0   rec: 0.0027835768963117608   f1: 0.005551700208188758
cluster: 13 pre: 1.0   rec: 0.003201024327784891   f1: 0.00638162093